In [7]:
import polars as pl
import numpy as np
import pandas as pd
from tqdm import tqdm  # <--- The loading bar library!

def rsi(prices, period: int = 14):
    delta = prices.diff()
    up = delta.clip(lower=0)
    down = -delta.clip(upper=0)
    ma_up = up.rolling(period).mean()
    ma_down = down.rolling(period).mean()
    rs = ma_up / ma_down
    return 100 - (100 / (1 + rs))

PATH = r"C:\Users\Jouke\Documents\evedata-logger\output\update_2025-07-13.csv"
df = pl.read_csv(PATH).to_pandas()
df['date'] = pd.to_datetime(df['date'])

rsi_windows = [7, 14, 21]
buy_threshs = [25, 30, 35]
sell_threshs = [65, 70, 75]

all_results = []
type_ids = df['type_id'].unique()

for item_id in tqdm(type_ids, desc="Scanning items"):
    item_df = df[df['type_id'] == item_id].sort_values('date').reset_index(drop=True)
    prices = item_df['average']
    if len(prices) < 50:
        continue
    split = int(len(prices) * 0.75)
    type_name = item_df['type_name'].iloc[0] if 'type_name' in item_df else str(item_id)

    for win in rsi_windows:
        rsi_series = rsi(prices, period=win)
        for buy_th in buy_threshs:
            for sell_th in sell_threshs:
                signals = (rsi_series < buy_th).astype(int) - (rsi_series > sell_th).astype(int)
                positions = signals.shift().fillna(0).replace(-1, 0).cumsum().clip(lower=0, upper=1)
                # IS
                IS_roi, IS_trades = 0, 0
                IS_pos = positions.iloc[:split]
                IS_trade_idx = IS_pos[IS_pos.diff().fillna(0) != 0].index.tolist()
                for i in range(0, len(IS_trade_idx)-1, 2):
                    buy = IS_trade_idx[i]
                    if i+1 < len(IS_trade_idx):
                        sell = IS_trade_idx[i+1]
                        buy_px, sell_px = prices.iloc[buy], prices.iloc[sell]
                        IS_roi += (sell_px - buy_px) / buy_px
                        IS_trades += 1
                # OOS
                OOS_roi, OOS_trades = 0, 0
                OOS_pos = positions.iloc[split:]
                OOS_trade_idx = OOS_pos[OOS_pos.diff().fillna(0) != 0].index.tolist()
                for i in range(0, len(OOS_trade_idx)-1, 2):
                    buy = OOS_trade_idx[i]
                    if i+1 < len(OOS_trade_idx):
                        sell = OOS_trade_idx[i+1]
                        buy_px, sell_px = prices.iloc[split + buy], prices.iloc[split + sell]
                        OOS_roi += (sell_px - buy_px) / buy_px
                        OOS_trades += 1

                all_results.append({
                    'type_id': item_id,
                    'type_name': type_name,
                    'win': win, 'buy_th': buy_th, 'sell_th': sell_th,
                    'IS_trades': IS_trades, 'IS_roi%': IS_roi*100,
                    'OOS_trades': OOS_trades, 'OOS_roi%': OOS_roi*100
                })

resdf = pd.DataFrame(all_results)
good = resdf[(resdf['OOS_trades'] >= 10) & (resdf['OOS_roi%'] >= 1)]

print("=== ALL ITEMS: BEST GRID RESULTS (OOS trades >=100 and ROI>=1%) ===")
if good.empty:
    print("No items met the filter. Try relaxing the trade/ROI thresholds or expanding grid.")
else:
    print(good.sort_values(['type_name', 'OOS_roi%'], ascending=[True, False]).groupby('type_name').head(3).to_string(index=False))

# Export all results, even those that didn't meet the filter
resdf.to_csv("eve_rsi_gridsearch_all_results.csv", index=False)

# Export only the gridsearch hits that met the strict filter
good.to_csv("eve_rsi_gridsearch_filtered_results.csv", index=False)

print("Exported:")
print(" - All grid search results → eve_rsi_gridsearch_all_results.csv")
print(" - Filtered (passing) results → eve_rsi_gridsearch_filtered_results.csv")



Scanning items: 100%|██████████| 14110/14110 [06:46<00:00, 34.73it/s] 


=== ALL ITEMS: BEST GRID RESULTS (OOS trades >=100 and ROI>=1%) ===
No items met the filter. Try relaxing the trade/ROI thresholds or expanding grid.
Exported:
 - All grid search results → eve_rsi_gridsearch_all_results.csv
 - Filtered (passing) results → eve_rsi_gridsearch_filtered_results.csv


In [8]:
import polars as pl
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

def rsi(prices, period: int = 14):
    delta = prices.diff()
    up = delta.clip(lower=0)
    down = -delta.clip(upper=0)
    ma_up = up.rolling(period).mean()
    ma_down = down.rolling(period).mean()
    rs = ma_up / ma_down
    return 100 - (100 / (1 + rs))

# Parameters
PATH = r"C:\Users\Jouke\Documents\evedata-logger\output\update_2025-07-13.csv"
EXPORT_DIR = r"C:\Users\Jouke\Documents\evedata-logger\output\plots"

# Make export directory if not exists
os.makedirs(EXPORT_DIR, exist_ok=True)

df = pl.read_csv(PATH).to_pandas()
df['date'] = pd.to_datetime(df['date'])

# RSI grid: aggressive short windows
rsi_windows = list(range(2, 15))         # 2–14 inclusive
buy_threshs = range(15, 41, 5)           # 15, 20, 25, 30, 35, 40
sell_threshs = range(60, 91, 5)          # 60, 65, 70, 75, 80, 85, 90

all_results = []
type_ids = df['type_id'].unique()

for item_id in tqdm(type_ids, desc="Scanning items"):
    item_df = df[df['type_id'] == item_id].sort_values('date').reset_index(drop=True)
    prices = item_df['average']
    if len(prices) < 50:
        continue
    split = int(len(prices) * 0.75)
    type_name = item_df['type_name'].iloc[0] if 'type_name' in item_df else str(item_id)

    for win in rsi_windows:
        rsi_series = rsi(prices, period=win)
        for buy_th in buy_threshs:
            for sell_th in sell_threshs:
                if buy_th >= sell_th:
                    continue
                signals = (rsi_series < buy_th).astype(int) - (rsi_series > sell_th).astype(int)
                positions = signals.shift().fillna(0).replace(-1, 0).cumsum().clip(lower=0, upper=1)
                # IS
                IS_roi, IS_trades = 0, 0
                IS_pos = positions.iloc[:split]
                IS_trade_idx = IS_pos[IS_pos.diff().fillna(0) != 0].index.tolist()
                for i in range(0, len(IS_trade_idx)-1, 2):
                    buy = IS_trade_idx[i]
                    if i+1 < len(IS_trade_idx):
                        sell = IS_trade_idx[i+1]
                        buy_px, sell_px = prices.iloc[buy], prices.iloc[sell]
                        IS_roi += (sell_px - buy_px) / buy_px
                        IS_trades += 1
                # OOS
                OOS_roi, OOS_trades = 0, 0
                OOS_pos = positions.iloc[split:]
                OOS_trade_idx = OOS_pos[OOS_pos.diff().fillna(0) != 0].index.tolist()
                for i in range(0, len(OOS_trade_idx)-1, 2):
                    buy = OOS_trade_idx[i]
                    if i+1 < len(OOS_trade_idx):
                        sell = OOS_trade_idx[i+1]
                        buy_px, sell_px = prices.iloc[split + buy], prices.iloc[split + sell]
                        OOS_roi += (sell_px - buy_px) / buy_px
                        OOS_trades += 1

                all_results.append({
                    'type_id': item_id,
                    'type_name': type_name,
                    'win': win, 'buy_th': buy_th, 'sell_th': sell_th,
                    'IS_trades': IS_trades, 'IS_roi%': IS_roi*100,
                    'OOS_trades': OOS_trades, 'OOS_roi%': OOS_roi*100
                })

resdf = pd.DataFrame(all_results)

# Filter for robust hits: OOS trades ≥30 and OOS ROI ≥5%
good = resdf[(resdf['OOS_trades'] >= 30) & (resdf['OOS_roi%'] >= 5)]

# Export results
all_path = os.path.join(EXPORT_DIR, "eve_rsi_gridsearch_all_results.csv")
good_path = os.path.join(EXPORT_DIR, "eve_rsi_gridsearch_filtered_results.csv")
resdf.to_csv(all_path, index=False)
good.to_csv(good_path, index=False)

print(f"\nExported all results to: {all_path}")
print(f"Exported filtered results to: {good_path}")
print(f"\n=== TOP FILTERED GRID RESULTS (OOS trades ≥30, ROI ≥5%) ===")
if good.empty:
    print("No items met the filter. Try relaxing the thresholds or inspecting all_results for hidden edge.")
else:
    print(good.sort_values(['type_name', 'OOS_roi%'], ascending=[True, False]).groupby('type_name').head(3).to_string(index=False))


Scanning items: 100%|██████████| 14110/14110 [2:01:19<00:00,  1.94it/s] 



Exported all results to: C:\Users\Jouke\Documents\evedata-logger\output\plots\eve_rsi_gridsearch_all_results.csv
Exported filtered results to: C:\Users\Jouke\Documents\evedata-logger\output\plots\eve_rsi_gridsearch_filtered_results.csv

=== TOP FILTERED GRID RESULTS (OOS trades ≥30, ROI ≥5%) ===
No items met the filter. Try relaxing the thresholds or inspecting all_results for hidden edge.
